In [1]:
import sys
sys.path.insert(0, "../..")

from MPyDATA_examples.Olesik_et_al_2020.analysis import compute_figure_data
from MPyDATA_examples.Olesik_et_al_2020.physics.equilibrium_drop_growth import PdfEvolver
from MPyDATA_examples.Olesik_et_al_2020.plotter import Plotter
from MPyDATA_examples.Olesik_et_al_2020.distributions import n_n, n_s
from MPyDATA_examples.Olesik_et_al_2020.coordinates import x_id, x_p2, x_p3,  x_log_of_pn
from MPyDATA_examples.utils.error_norms import L2
from MPyDATA_examples.utils.show_plot import show_plot

import matplotlib.pyplot as plt
import matplotlib

import numpy as np

In [2]:
def compute_and_plot(psi_coord, grid_layout, n_bins, GC_max, opt_set, plots, filename):
    results, setup = compute_figure_data(
        psi_coord=psi_coord, 
        grid_layouts=(grid_layout,),
        nr=n_bins,
        GC_max = GC_max,
        opt_set=opt_set
    )
    
    colors = ['red', 'green', 'purple', 'blue', 'orange']
    matplotlib.rcParams.update({'font.size': 16})
    for coord in results.keys():
        out_steps = results[coord]['grid']['out_steps']
        dt = results[coord]['grid']['dt']
        plotter = Plotter(setup, plots=plots)
        for opt_i, opts in enumerate(results[coord]['numerical'].keys()): 
            plot_data = results[coord]['numerical'][opts]
            for i in range(len(out_steps)):
                mnorm = setup.mixing_ratios[i]
                t = out_steps[i] * dt
                linewidth = 1+1.5*i/len(out_steps)
                if opt_i == 0:
                    plotter.pdf_curve(PdfEvolver(setup.pdf, setup.drdt, t), mnorm)
                    plotter.pdf_histogram(
                        results[coord]['grid']['r'], 
                        results[coord]['analytical'][i], 
                        bin_boundaries = results[coord]['grid']['rh'], 
                        label='analytical', 
                        mnorm=mnorm,
                        color='black',
                        linewidth = linewidth
                    )
                str_repl = [["'n_iters': 1","upwind"],
                            ["'n_iters': 2","MPDATA 2 iterations"],
                            ["'n_iters': 3","MPDATA 3 iterations"],
                            ["'fct': True","+ fct"],
                            ["'iga': True","+ iga"],
                            ["'tot': True","+ tot"],
                            ["{",""],["}",""],[","," "]]
                            
                for repl in str_repl:
                    opts = opts.replace(repl[0], repl[1])
                plotter.pdf_histogram(
                    results[coord]['grid']['r'], 
                    plot_data[i], 
                    label=opts, 
                    bin_boundaries=results[coord]['grid']['rh'], 
                    linewidth = linewidth,
                    mnorm=mnorm, color = colors[opt_i]
                )
        print(f"grid_layout: {coord}")
        show_plot(filename)
        

In [3]:
plot_setup = {"psi_coord":x_p2(), "grid_layout":x_log_of_pn(base=2), "GC_max":.26,"n_bins":75}
plot_setup2 = {"psi_coord":x_id(), "grid_layout":x_id(), "GC_max":.07,"n_bins":75}

In [4]:
compute_and_plot(**plot_setup, opt_set=({"n_iters":1},), plots=('n','m'), filename = 'fig_upwinda.pdf')

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.


CacheInfo(hits=0, misses=1, maxsize=128, currsize=1)


[Parallel(n_jobs=-2)]: Done   1 tasks      | elapsed:   12.8s
[Parallel(n_jobs=-2)]: Done   1 out of   1 | elapsed:   12.8s finished


grid_layout: x_log_of_pn


In [5]:
compute_and_plot(**plot_setup2, opt_set=({"n_iters":1},), plots=['n','m'],  filename = 'fig_upwindb.pdf')

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.


CacheInfo(hits=1, misses=1, maxsize=128, currsize=1)


[Parallel(n_jobs=-2)]: Done   1 tasks      | elapsed:   11.4s
[Parallel(n_jobs=-2)]: Done   1 out of   1 | elapsed:   11.4s finished


grid_layout: x_id


In [6]:
compute_and_plot(**plot_setup, opt_set=({'n_iters': 1},{'n_iters':2},{'n_iters':3}), plots=['n'], filename = 'fig_mpdatas.pdf')

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.


CacheInfo(hits=1, misses=2, maxsize=128, currsize=2)
CacheInfo(hits=1, misses=3, maxsize=128, currsize=3)
CacheInfo(hits=2, misses=3, maxsize=128, currsize=3)


ValueError: f(a) and f(b) must have different signs

In [ ]:
compute_and_plot(**plot_setup, opt_set=({'n_iters': 1},{'n_iters':2},{'n_iters':2,'infinite_gauge':True}), plots=['n'], filename = 'fig_iga.pdf')

In [ ]:
compute_and_plot(**plot_setup, opt_set=({'n_iters': 1},{'n_iters':2},{'n_iters':2,'infinite_gauge':True,'flux_corrected_transport':True}), plots=['n'], filename = 'fig_fct.pdf')

In [ ]:
compute_and_plot(**plot_setup, opt_set=({'n_iters': 1},{'n_iters':2},{'n_iters':2,'third_order_terms':True}), plots=['n'], filename = 'fig_tot.pdf')

In [ ]:
compute_and_plot(**plot_setup, opt_set=(
    {'n_iters':1},
    {'n_iters':2},
    {'n_iters':3,'infinite_gauge':True, 'flux_corrected_transport':True,'third_order_terms':True},
), plots=['n'], filename='fig_multiopt.pdf')